In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
%cd /content/drive/My\ Drive/PGDBA\ Academics/IIM/EMABE

/content/drive/My Drive/PGDBA Academics/IIM/EMABE


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
df = pd.read_csv('FimaNfipClaims.csv')

/tmp/ipython-input-20-631711394.py:1: DtypeWarning: Columns (7,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('FimaNfipClaims.csv')


In [21]:
# prompt: Display the missing count columns wise and remove the columns which has more than 50% missing values

# Display missing values per column
print(df.isnull().sum())

# Calculate missing value percentage per column
missing_percentage = df.isnull().sum() / len(df)

# Identify columns with more than 50% missing values
columns_to_drop = missing_percentage[missing_percentage > 0.5].index

# Drop the identified columns
df_cleaned = df.drop(columns=columns_to_drop)

print(f"\nOriginal shape: {df.shape}")
print(f"Shape after dropping columns: {df_cleaned.shape}")

# Display missing values after dropping columns
print("\nMissing values after dropping columns:")
print(df_cleaned.isnull().sum())

agricultureStructureIndicator            0
asOfDate                           2695373
basementEnclosureCrawlspaceType    1893514
policyCount                              0
crsClassificationCode              2579081
                                    ...   
censusTract                         137434
censusBlockGroupFips                137434
latitude                             41325
longitude                            41325
id                                       0
Length: 73, dtype: int64

Original shape: (2712269, 73)
Shape after dropping columns: (2712269, 54)

Missing values after dropping columns:
agricultureStructureIndicator                       0
policyCount                                         0
dateOfLoss                                          0
elevatedBuildingIndicator                           0
ratedFloodZone                                 139030
houseWorship                                        0
locationOfContents                             917821
numberOfF

In [25]:
# prompt: Display data where originalConstructionDate is recent than dateofLoss

# Convert date columns to datetime objects
df_cleaned['originalConstructionDate'] = pd.to_datetime(df_cleaned['originalConstructionDate'], errors='coerce')
df_cleaned['dateOfLoss'] = pd.to_datetime(df_cleaned['dateOfLoss'], errors='coerce')

# Filter data where originalConstructionDate is more recent than dateofLoss
recent_construction_loss_df = df_cleaned[df_cleaned['originalConstructionDate'] > df_cleaned['dateOfLoss']]

# Display the filtered data
print("\nData where originalConstructionDate is more recent than dateofLoss:")
recent_construction_loss_df[['dateOfLoss', 'originalConstructionDate']]


Data where originalConstructionDate is more recent than dateofLoss:


,dateOfLoss,originalConstructionDate
122,2012-10-29 00:00:00+00:00,2014-03-18 00:00:00+00:00
221,2017-04-18 00:00:00+00:00,2019-06-07 00:00:00+00:00
430,1991-10-31 00:00:00+00:00,1992-07-01 00:00:00+00:00
451,2005-08-29 00:00:00+00:00,2013-07-29 00:00:00+00:00
470,1980-04-15 00:00:00+00:00,1989-07-01 00:00:00+00:00
...,...,...
2711644,2012-10-29 00:00:00+00:00,2014-07-25 00:00:00+00:00
2711651,2005-08-29 00:00:00+00:00,2010-07-20 00:00:00+00:00
2711707,2012-10-29 00:00:00+00:00,2012-10-30 00:00:00+00:00
2711910,2012-10-29 00:00:00+00:00,2014-04-01 00:00:00+00:00


In [7]:
# prompt: Remove rows which has missing values

# Remove rows with any missing values
df_cleaned_rows = df_cleaned.dropna()

print(f"\nShape after dropping rows with missing values: {df_cleaned_rows.shape}")

# Display missing values after dropping rows
print("\nMissing values after dropping rows:")
print(df_cleaned_rows.isnull().sum())


Shape after dropping rows with missing values: (243694, 54)

Missing values after dropping rows:
agricultureStructureIndicator                 0
policyCount                                   0
dateOfLoss                                    0
elevatedBuildingIndicator                     0
ratedFloodZone                                0
houseWorship                                  0
locationOfContents                            0
numberOfFloorsInTheInsuredBuilding            0
nonProfitIndicator                            0
obstructionType                               0
occupancyType                                 0
originalConstructionDate                      0
originalNBDate                                0
amountPaidOnBuildingClaim                     0
amountPaidOnContentsClaim                     0
amountPaidOnIncreasedCostOfComplianceClaim    0
postFIRMConstructionIndicator                 0
rateMethod                                    0
smallBusinessIndicatorBuilding        

In [8]:
# prompt: Replace col 'replacementCostBasis' with 1 or 0

# Check unique values in the 'replacementCostBasis' column
print("\nUnique values in 'replacementCostBasis' before replacement:")
print(df_cleaned_rows['replacementCostBasis'].unique())

# Replace 'None' and 'Unknown' with 0 and any other value with 1
df_cleaned_rows['replacementCostBasis'] = df_cleaned_rows['replacementCostBasis'].apply(
    lambda x: 0 if x in ['A'] else 1
)

# Check unique values after replacement
print("\nUnique values in 'replacementCostBasis' after replacement:")
print(df_cleaned_rows['replacementCostBasis'].unique())


Unique values in 'replacementCostBasis' before replacement:
['R' 'A']

Unique values in 'replacementCostBasis' after replacement:
[1 0]


/tmp/ipython-input-8-1494690263.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned_rows['replacementCostBasis'] = df_cleaned_rows['replacementCostBasis'].apply(


In [9]:
# prompt: Create a column for age of Building by subtracting orginalConstructionDate from dateOfLoss

# Convert date columns to datetime objects
df_cleaned_rows['originalConstructionDate'] = pd.to_datetime(df_cleaned_rows['originalConstructionDate'], errors = 'coerce')
df_cleaned_rows['dateOfLoss'] = pd.to_datetime(df_cleaned_rows['dateOfLoss'], errors = 'coerce')

# Calculate the age of the building in years
df_cleaned_rows['BuildingAge'] = (df_cleaned_rows['dateOfLoss'] - df_cleaned_rows['originalConstructionDate']).dt.days / 365.25

# Display the first few rows with the new column
print("\nDataFrame with 'BuildingAge' column:")
print(df_cleaned_rows[['originalConstructionDate', 'dateOfLoss', 'BuildingAge']].head())


DataFrame with 'BuildingAge' column:
    originalConstructionDate                dateOfLoss  BuildingAge
3  1978-01-01 00:00:00+00:00 2017-08-26 00:00:00+00:00    39.649555
5  1970-07-01 00:00:00+00:00 1998-08-26 00:00:00+00:00    28.153320
41 1969-01-01 00:00:00+00:00 2005-04-04 00:00:00+00:00    36.254620
53 1985-05-14 00:00:00+00:00 2008-09-13 00:00:00+00:00    23.334702
62 1983-01-01 00:00:00+00:00 1992-06-26 00:00:00+00:00     9.483915


/tmp/ipython-input-9-1812685091.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned_rows['originalConstructionDate'] = pd.to_datetime(df_cleaned_rows['originalConstructionDate'], errors = 'coerce')
/tmp/ipython-input-9-1812685091.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned_rows['dateOfLoss'] = pd.to_datetime(df_cleaned_rows['dateOfLoss'], errors = 'coerce')
/tmp/ipython-input-9-1812685091.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [10]:
# prompt: List non numerical columns present in data

# Select only object (non-numerical) columns
non_numerical_columns = df_cleaned_rows.select_dtypes(include=['object']).columns

print("\nNon-numerical columns:")
print(list(non_numerical_columns))


Non-numerical columns:
['ratedFloodZone', 'originalNBDate', 'rateMethod', 'buildingDeductibleCode', 'causeOfDamage', 'condominiumCoverageTypeCode', 'contentsDeductibleCode', 'floodEvent', 'state', 'reportedCity', 'id']


In [11]:
col_to_remove = ['id', 'reportedCity', 'state','originalNBDate','dateOfLoss', 'rateMethod','floodEvent', 'ratedFloodZone', 'buildingDeductibleCode','causeOfDamage', 'condominiumCoverageTypeCode', 'contentsDeductibleCode']

In [12]:
# prompt: delete the col_to_remove from the dataframe

df_final = df_cleaned_rows.drop(columns=col_to_remove)

print(f"\nShape after removing specified columns: {df_final.shape}")

# Display missing values after removing specified columns
print("\nMissing values after removing specified columns:")
print(df_final.isnull().sum())

# Display the first few rows of the final DataFrame
print("\nFirst 5 rows of the final DataFrame:")
print(df_final.head())


Shape after removing specified columns: (243694, 43)

Missing values after removing specified columns:
agricultureStructureIndicator                 0
policyCount                                   0
elevatedBuildingIndicator                     0
houseWorship                                  0
locationOfContents                            0
numberOfFloorsInTheInsuredBuilding            0
nonProfitIndicator                            0
obstructionType                               0
occupancyType                                 0
originalConstructionDate                      2
amountPaidOnBuildingClaim                     0
amountPaidOnContentsClaim                     0
amountPaidOnIncreasedCostOfComplianceClaim    0
postFIRMConstructionIndicator                 0
smallBusinessIndicatorBuilding                0
totalBuildingInsuranceCoverage                0
totalContentsInsuranceCoverage                0
yearOfLoss                                    0
primaryResidenceIndicator       

In [13]:
# prompt: One_hot encode the non_numerical datatypes with 1 or 0 and not true or false

# Select non-numerical columns again from the df_final DataFrame
non_numerical_columns_final = df_final.select_dtypes(include=['object']).columns

print("\nNon-numerical columns in df_final:")
print(list(non_numerical_columns_final))

# Apply one-hot encoding using pandas get_dummies
# Set dtype to int to get 0 and 1 instead of False and True
df_encoded = pd.get_dummies(df_final, columns=non_numerical_columns_final, dtype=int)

print("\nShape after one-hot encoding:")
print(df_encoded.shape)

print("\nFirst 5 rows after one-hot encoding:")
print(df_encoded.head())

# Verify that all columns are now numeric or the new one-hot encoded columns
print("\nData types after one-hot encoding:")
df_encoded.dtypes


Non-numerical columns in df_final:
[]

Shape after one-hot encoding:
(243694, 43)

First 5 rows after one-hot encoding:
    agricultureStructureIndicator  policyCount  elevatedBuildingIndicator  \
3                               0            1                          1   
5                               0            1                          0   
41                              0            1                          0   
53                              0            1                          1   
62                              0            1                          0   

    houseWorship  locationOfContents  numberOfFloorsInTheInsuredBuilding  \
3              0                 3.0                                 1.0   
5              0                 4.0                                 2.0   
41             0                 4.0                                 2.0   
53             0                 3.0                                 1.0   
62             0                 3.0

,0
agricultureStructureIndicator,int64
policyCount,int64
elevatedBuildingIndicator,int64
houseWorship,int64
locationOfContents,float64
numberOfFloorsInTheInsuredBuilding,float64
nonProfitIndicator,int64
obstructionType,float64
occupancyType,float64
originalConstructionDate,"datetime64[ns, UTC]"


In [14]:
# prompt: remove the columns of objects datatype

# Identify columns with 'object' datatype
object_columns = df_encoded.select_dtypes(include=['object']).columns

# Drop columns with 'object' datatype
df_numeric = df_encoded.drop(columns=object_columns)

print(f"\nShape after removing object columns: {df_numeric.shape}")

print("\nData types after removing object columns:")
print(df_numeric.dtypes)

print("\nFirst 5 rows after removing object columns:")
print(df_numeric.head())


Shape after removing object columns: (243694, 43)

Data types after removing object columns:
agricultureStructureIndicator                               int64
policyCount                                                 int64
elevatedBuildingIndicator                                   int64
houseWorship                                                int64
locationOfContents                                        float64
numberOfFloorsInTheInsuredBuilding                        float64
nonProfitIndicator                                          int64
obstructionType                                           float64
occupancyType                                             float64
originalConstructionDate                      datetime64[ns, UTC]
amountPaidOnBuildingClaim                                 float64
amountPaidOnContentsClaim                                 float64
amountPaidOnIncreasedCostOfComplianceClaim                float64
postFIRMConstructionIndicator                   

In [15]:
# prompt: Remove original construction date column from the dtaframe

col_to_remove = ['originalConstructionDate','netBuildingPaymentAmount',
 'netContentsPaymentAmount']

df_final = df_numeric.drop(columns=col_to_remove)

print(f"\nShape after removing specified columns: {df_final.shape}")

# Display missing values after removing specified columns
print("\nMissing values after removing specified columns:")
print(df_final.isnull().sum())

# Display the first few rows of the final DataFrame
print("\nFirst 5 rows of the final DataFrame:")
print(df_final.head())

# Select non-numerical columns again from the df_final DataFrame
non_numerical_columns_final = df_final.select_dtypes(include=['object']).columns

print("\nNon-numerical columns in df_final:")
print(list(non_numerical_columns_final))




Shape after removing specified columns: (243694, 40)

Missing values after removing specified columns:
agricultureStructureIndicator                 0
policyCount                                   0
elevatedBuildingIndicator                     0
houseWorship                                  0
locationOfContents                            0
numberOfFloorsInTheInsuredBuilding            0
nonProfitIndicator                            0
obstructionType                               0
occupancyType                                 0
amountPaidOnBuildingClaim                     0
amountPaidOnContentsClaim                     0
amountPaidOnIncreasedCostOfComplianceClaim    0
postFIRMConstructionIndicator                 0
smallBusinessIndicatorBuilding                0
totalBuildingInsuranceCoverage                0
totalContentsInsuranceCoverage                0
yearOfLoss                                    0
primaryResidenceIndicator                     0
buildingDamageAmount            

In [16]:
# prompt: Remove the rows with missing values from the dataframe

# Remove rows with any missing values
df_final.dropna(inplace=True)


In [17]:
# prompt: Pass this statsmodel for linear regression and show the column that has significance

import statsmodels.api as sm

# Define the dependent variable (y) and independent variables (X)
# Assuming 'amountPaidOnBuildingClaim' is the target variable you want to predict
y = df_final['amountPaidOnBuildingClaim']
X = df_final.drop('amountPaidOnBuildingClaim', axis=1)

# Add a constant to the independent variables matrix (for the intercept)
X = sm.add_constant(X)

# Fit the linear regression model using OLS (Ordinary Least Squares)
model = sm.OLS(y, X).fit()

# Print the summary of the regression results
print(model.summary())

# Extract the p-values
p_values = model.pvalues

# Set a significance level (commonly 0.05)
alpha = 0.05

# Identify columns with p-value less than alpha (statistically significant)
significant_columns = p_values[p_values < alpha].index.tolist()

print(f"\nColumns with statistical significance (p < {alpha}):")
significant_columns


                                OLS Regression Results                               
Dep. Variable:     amountPaidOnBuildingClaim   R-squared:                       0.392
Model:                                   OLS   Adj. R-squared:                  0.392
Method:                        Least Squares   F-statistic:                     4128.
Date:                       Thu, 19 Jun 2025   Prob (F-statistic):               0.00
Time:                               05:10:13   Log-Likelihood:            -2.9528e+06
No. Observations:                     243692   AIC:                         5.906e+06
Df Residuals:                         243653   BIC:                         5.906e+06
Df Model:                                 38                                         
Covariance Type:                   nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------

['const',
 'policyCount',
 'elevatedBuildingIndicator',
 'houseWorship',
 'locationOfContents',
 'nonProfitIndicator',
 'obstructionType',
 'occupancyType',
 'amountPaidOnContentsClaim',
 'amountPaidOnIncreasedCostOfComplianceClaim',
 'postFIRMConstructionIndicator',
 'smallBusinessIndicatorBuilding',
 'totalBuildingInsuranceCoverage',
 'totalContentsInsuranceCoverage',
 'yearOfLoss',
 'primaryResidenceIndicator',
 'buildingDamageAmount',
 'buildingPropertyValue',
 'ficoNumber',
 'floodWaterDuration',
 'iccCoverage',
 'nfipRatedCommunityNumber',
 'numberOfUnits',
 'buildingReplacementCost',
 'waterDepth',
 'rentalPropertyIndicator',
 'reportedZipCode',
 'countyCode',
 'latitude',
 'longitude',
 'BuildingAge']

In [18]:
df_final.to_csv('final.csv')